# Langton's ant

[Background](http://mathworld.wolfram.com/LangtonsAnt.html)

Using this cellular automaton to demonstrate

* Python generators
* `defaultdict`
* `ipythonblocks`
* Basic notebook interactivity

In [1]:
# Convenience functions for turning and moving
def cw(orientation):
    return (orientation[1],-orientation[0])
def acw(orientation):
    return (-orientation[1],orientation[0])
def move(position, orientation):
    return (position[0]+orientation[0], position[1]+orientation[1])

In [2]:
acw(acw((-1,0)))

(1, 0)

In [3]:
from collections import defaultdict

In [4]:
from ipythonblocks import BlockGrid

A generator is a good choice. Each `next` will move the ant once.

I've used a `defaultdict` as the underlying structure, the keys being coordinate pairs and the values being `0` for white and `1` for black. This allows the grid to grow indefinitely, and respond with a default of `0` for coordinates the ant hasn't visited yet, making it easier to draw later.

In [5]:
def langtongenerator():
    
    langtongrid = defaultdict(int)
    position = (0,0)
    orientation = (0,1)
    while True:
        # If cell is white
        if langtongrid[position] == 0:
            # Turn it black
            langtongrid[position] = 1
            # Rotate clockwise
            orientation = cw(orientation)
        # If cell is black
        else:
            # Turn it white
            langtongrid[position] = 0
            # Rotate anticlockwise
            orientation = acw(orientation)
        # Move
        position = move(position, orientation)
        yield langtongrid

In [6]:
lg = langtongenerator()

In [7]:
next(lg)

defaultdict(int, {(0, 0): 1})

In [8]:
def drawgrid(langtongrid,blocksize=20):
    minx = min([i for (i,_) in langtongrid.keys()])
    maxx = max([i for (i,_) in langtongrid.keys()])
    miny = min([i for (_,i) in langtongrid.keys()])
    maxy = max([i for (_,i) in langtongrid.keys()])
    grid = BlockGrid(maxx-minx+1,maxy-miny+1,block_size=blocksize)
    for block in grid:
        y = block.row+miny
        x = block.col+minx
        if langtongrid[(x, y)] == 0:
            block.rgb = (128,128,128)
        else:
            block.rgb = (0,0,0)
    return grid

In [9]:
drawgrid(next(lg))

,


Now to make it slightly more interactive.

In [10]:
import ipywidgets as widgets

In [11]:
lg = langtongenerator()

def fn():
    drawgrid(next(lg)).show()

widgets.interact_manual(fn, text="Move the ant");

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

It gets tiring hitting that button over and over though, so let's use a `play` widget.

In [12]:
lg = langtongenerator()
def fn(p):
    return drawgrid(next(lg),blocksize=5)
# This will play 100 iterations. Hit the loop button if you just want to keep going
play = widgets.Play(value=0,min=0,max=100,step=1,interval=0)
widgets.interact(fn,p=play);

interactive(children=(Play(value=0, description='p', interval=0), Output()), _dom_classes=('widget-interact',))

This could be made faster by only redrawing the parts of the grid affected each iteration but I like having the generating and drawing code isolated.